<a href="https://colab.research.google.com/github/abisubramanya27/CS6910_Assignment3/blob/master/src/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"

--2021-04-28 21:03:32--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  39.2MB/s    in 18s     

2021-04-28 21:03:50 (107 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [2]:
!tar xopf dakshina_dataset_v1.0.tar

In [3]:
!ls dakshina_dataset_v1.0/hi/lexicons

hi.translit.sampled.dev.tsv   hi.translit.sampled.train.tsv
hi.translit.sampled.test.tsv


In [4]:
import numpy as np

def read_data(data_path, characters = False):
    # Returns the (input, output) pair from the dataset
    # If characters == True, the input/output would be in the form list of characters, else as string

    with open(data_path, "r", encoding="utf-8") as f:
        lines = [line.split("\t") for line in f.read().split("\n") if line != '']
    
    input, target = [val[1] for val in lines], [val[0] for val in lines]
    if characters:
        input, target = [list(inp_str) for inp_str in input], [list(tar_str) for tar_str in target]
    return input, target


def process_data(input, enc_timesteps, input_char_enc, target = None, dec_timesteps = None, target_char_enc = None):
    # Returns the input and target data in a form needed by the Keras embedding layer, where each character is encoded by an integer

    # ' ' -- space (equivalent to no meaningful input)
    encoder_input = np.array([[input_char_enc[ch] for ch in string] + [input_char_enc[' ']] * (enc_timesteps - len(string)) for string in input])

    decoder_input, decoder_target = None, None
    if target is not None and dec_timesteps is not None and target_char_enc is not None:
        # '\t' -- start of sequence, '\n' -- end of sequence
        decoder_input = np.array([[target_char_enc['\t']] + [target_char_enc[ch] for ch in string] + [target_char_enc['\n']] 
                                    + [target_char_enc[' ']] * (dec_timesteps - len(string) - 2) for string in target])
        decoder_target = np.zeros((decoder_input.shape[0], dec_timesteps, len(target_char_enc)), dtype='float32')

        for i in range(decoder_input.shape[0]):
            for t, char_ind in enumerate(decoder_input[i]):
                if t > 0:
                    decoder_target[i,t-1,char_ind] = 1.0
            decoder_target[i,t:,target_char_enc[' ']] = 1.0

    return encoder_input, decoder_input, decoder_target


def encode_decode_characters(train_input, train_target, val_input, val_target):
    # Returns the encoder for characters to integer (as a dictionary) and decoder for integers to characters (as a list) for input and target data

    input_char_enc = {}
    input_char_dec = []
    max_encoder_seq_length = 1
    for string in train_input + val_input:
        max_encoder_seq_length = max(max_encoder_seq_length, len(string))
        for char in string:
            if char not in input_char_enc:
                input_char_enc[char] = len(input_char_dec)
                input_char_dec.append(char)
    if ' ' not in input_char_enc:
        input_char_enc[' '] = len(input_char_dec)
        input_char_dec.append(' ')

    target_char_enc = {}
    target_char_dec = []
    target_char_enc['\t'] = len(target_char_dec)
    target_char_dec.append('\t')
    max_decoder_seq_length = 1
    for string in train_target + val_target:
        max_decoder_seq_length = max(max_decoder_seq_length, len(string)+2)
        for char in string:
            if char not in target_char_enc:
                target_char_enc[char] = len(target_char_dec)
                target_char_dec.append(char)
    target_char_enc['\n'] = len(target_char_dec)
    target_char_dec.append('\n')
    if ' ' not in target_char_enc:
        target_char_enc[' '] = len(target_char_dec)
        target_char_dec.append(' ')

    print("Number of training samples:", len(train_input))
    print("Number of validation samples:", len(val_input))
    print("Number of unique input tokens:", len(input_char_dec))
    print("Number of unique output tokens:", len(target_char_dec))
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)

    return input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length

def max_seq_length(data):
    max_len = 0
    for word in data:
        max_len = max(max_len, len(word))
    
    return max_len


In [5]:
input_char_enc = {}
input_char_dec = []
target_char_enc = {}
target_char_dec = []
max_encoder_seq_length = 0
max_decoder_seq_length = 0

# Reading training and validation data
train_inp, train_out = read_data('./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv')
val_inp, val_out = read_data('./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv')
# Assigning encoder and decoder for input and target characters
input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length = encode_decode_characters(
    train_inp, train_out, val_inp, val_out)

# Assigning training and validation encoder input, decoder input, decoder output
train_enc_input, train_dec_input, train_dec_target = process_data(train_inp, max_encoder_seq_length, input_char_enc, train_out, 
                                                                  max_decoder_seq_length, target_char_enc)
val_enc_input, val_dec_input, val_dec_target = process_data(val_inp, max_encoder_seq_length, input_char_enc, val_out, 
                                                            max_decoder_seq_length, target_char_enc)


Number of training samples: 44204
Number of validation samples: 4358
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [6]:
import tensorflow as tf
import tensorflow.keras as keras

def create_model(encoder_vocab_size, decoder_vocab_size, inp_emb_size=16, no_enc_layers=1, no_dec_layers=1, 
                 hid_layer_size=32, cell_type='LSTM', dropout=0, r_dropout=0, cell_activation='tanh'):
    
    get_cell = {
        'RNN': keras.layers.SimpleRNN,
        'GRU': keras.layers.GRU,
        'LSTM': keras.layers.LSTM
    }
    # Encoder input and embedding
    encoder_input = keras.layers.Input(shape=(None,), name="input_1")
    encoder_inp_emb = keras.layers.Embedding(encoder_vocab_size, inp_emb_size, name="embedding_1")(encoder_input)

    # Encoder cell layers
    encoder_seq, *encoder_state = get_cell[cell_type](hid_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                                      dropout=dropout, recurrent_dropout=r_dropout, name="encoder_1")(
                                                            encoder_inp_emb
                                                     )
    for i in range(1, no_enc_layers):
        encoder_seq, *encoder_state = get_cell[cell_type](hid_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                                          dropout=dropout, recurrent_dropout=r_dropout, name="encoder_"+str(i+1))(
                                                                encoder_seq
                                                         )
    
    # Decoder input and embedding
    decoder_input = keras.layers.Input(shape=(None,), name="input_2")
    decoder_inp_emb = keras.layers.Embedding(decoder_vocab_size, inp_emb_size, name="embedding_2")(decoder_input)

    # Decoder cell layers
    decoder_seq, *_ = get_cell[cell_type](hid_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                          dropout=dropout, recurrent_dropout=r_dropout, name="decoder_1")(
                                                decoder_inp_emb, initial_state=encoder_state
                                         )
    for i in range(1, no_dec_layers):
        decoder_seq, *_ = get_cell[cell_type](hid_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                              dropout=dropout, recurrent_dropout=r_dropout, name="decoder_"+str(i+1))(
                                                    decoder_seq, initial_state=encoder_state
                                             )
    
    # Softmax FC layer
    decoder_dense_output = keras.layers.Dense(decoder_vocab_size, activation="softmax", name="dense_1")(
        decoder_seq
    )

    # Define the model that will turn encoder_input_data and decoder_input_data into decoder_target_data
    model = keras.Model([encoder_input, decoder_input], decoder_dense_output)

    model.summary()
    return model

In [7]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 8.8MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 163kB 35.2MB/s 
     |████████████████████████████████| 133kB 33.5MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=137a37e557a3d52e347a158afacf0e42b176d506bed842d0220a39a7e71374b3
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=71d9a639023ebbf0ea01683aaa84d32c3fe50a2c00e55bae32e7cc4e4fe00f8a
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

def train_model(model, train_input_data, train_target_data, val_input_data, val_target_data, 
                batch_size=64, optimizer = 'adam', learning_rate = 1e-3, epochs = 10, loss_fn = 'categorical_crossentropy'):
    # Function to train the model using the mentioned optimizer, learning rate and epochs using given training and validation data

    if optimizer == 'adam':
        model.compile(optimizer = Adam(learning_rate=learning_rate), loss = loss_fn, metrics = ['accuracy'])
    elif optimizer == 'momentum':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9), loss = loss_fn, metrics = ['accuracy'])
    elif optimizer == 'rmsprop':
        model.compile(optimizer = RMSprop(learning_rate=learning_rate), loss = loss_fn, metrics = ['accuracy'])
    elif optimizer == 'nesterov':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9, nesterov = True), loss = loss_fn, metrics = ['accuracy'])
    elif optimizer == 'nadam':
        model.compile(optimizer = Nadam(learning_rate=learning_rate), loss = loss_fn, metrics = ['accuracy'])
    else:
        model.compile(optimizer = SGD(learning_rate=learning_rate), loss = loss_fn, metrics = ['accuracy'])

    # Using validation accuracy as the metric to monitor as that is what is intended to be maximized
    model.fit(train_input_data,
              train_target_data,
              batch_size = batch_size,
              epochs = epochs, 
              validation_data = (val_input_data, val_target_data),
              verbose = 2,
              callbacks = [WandbCallback(monitor='val_accuracy'), EarlyStopping(monitor='val_accuracy', patience=5)])

    return model

In [9]:
config_1 = {
    "learning_rate": 1e-3,                                      # Hyperparameter for updating the parameters in gradient descent
    "epochs": 10,                                               # Number of epochs to train the model   
    "optimizer": 'adam',                                    # Gradient descent algorithm used for the parameter updation
    "batch_size": 64,                                           # Batch size used for the optimizer
    "loss_function": 'categorical_crossentropy',                # Loss function used in the optimizer
    "architecture": 'RNN',                                      # Type of neural network used
    "dataset": "Dakshina",                                      # Name of dataset
    "inp_emb_size": 16,                                         # Number of filters for the first convolution layer
    "no_enc_layers": 1,                                         # The factor by which the number of filters change in the subseqeuent convolution layers
    "no_dec_layers": 1,                                         # Number of neurons in the dense FC layer
    "hid_layer_size": 16,                                       # True : Data augmentation is done during training, False : No data augmentation done
    "dropout" : 0.25,                                           # Probability of dropping out a neuron in dropout technique
    "cell_type": 'RNN',
    "beam": 1,
}


# run = wandb.init(project="assignment3", entity="abisheks", reinit=True)
# tf.keras.backend.clear_session()
# model = create_model(len(input_char_dec), len(target_char_dec))
# model = train_model(model, [train_enc_input,train_dec_input], train_dec_target, [val_enc_input,val_dec_input], val_dec_target)
# run.finish()

def RNN_train(config):
  model = create_model(len(input_char_dec), len(target_char_dec), config['inp_emb_size'], config['no_enc_layers'], 
                       config['no_dec_layers'], config['hid_layer_size'], config['cell_type'], config['dropout'], config['dropout'])
  
  model = train_model(model, [train_enc_input,train_dec_input], train_dec_target, [val_enc_input,val_dec_input], val_dec_target, 
                      config['batch_size'], config['optimizer'], config['learning_rate'], config['epochs'], config['loss_function'])
  return model

# run = wandb.init(project="assignment3", entity="abisheks", reinit=True, config=config_1)
# model = RNN_train(config_1)
# run.finish()

In [10]:
# print([layer.__class__.__name__ for layer in model.layers])
# print(model.input)
# en1, *en2 = model.get_layer("encoder_1").output
# print(en2)

In [ ]:
def create_inference_model(model):
    no_enc_layers, no_dec_layers = 0, 0
    for layer in model.layers:
        no_enc_layers += layer.name.startswith('encoder')
        no_dec_layers += layer.name.startswith('decoder')

    # Encoder input
    encoder_input = model.input[0]      # Input_1
    # Encoder cell final layer
    encoder_cell = model.get_layer("encoder_"+str(no_enc_layers))
    encoder_type = encoder_cell.__class__.__name__
    encoder_seq, *encoder_state = encoder_cell.output
    # Encoder model
    encoder_model = keras.Model(encoder_input, encoder_state)

    # Decoder input
    decoder_input = model.input[1]      # Input_2
    decoder_inp_emb = model.get_layer("embedding_1")(decoder_input)
    decoder_seq = decoder_inp_emb
    # Inputs to decoder layers' initial states
    decoder_states, decoder_state_inputs = [], []
    for i in range(1, no_dec_layers+1):
        if encoder_type == 'LSTM':
            decoder_state_input = [keras.Input(shape=(encoder_state[0].shape[1],), name="input_"+str(2*i+1)), 
                                   keras.Input(shape=(encoder_state[1].shape[1],), name="input_"+str(2*i+2))]
        else:
            decoder_state_input = [keras.Input(shape=(encoder_state[0].shape[1],), name="input_"+str(i+2))]

        decoder_cell = model.get_layer("decoder_"+str(i))
        decoder_seq, *decoder_state = decoder_cell(decoder_seq, initial_state=decoder_state_input)
        decoder_states += decoder_state
        decoder_state_inputs += decoder_state_input

    # Softmax FC layer
    decoder_dense = model.get_layer("dense")
    decoder_dense_output = decoder_dense(decoder_seq)

    # Decoder model
    decoder_model = keras.Model(
        [decoder_input] + decoder_state_inputs, [decoder_dense_output] + decoder_states
    )

    return encoder_model, decoder_model, no_enc_layers, no_dec_layers


def infer(model, input_seqs, target_char_enc, target_char_dec, max_decoder_timesteps):
    encoder_model, decoder_model, no_enc_layers, no_dec_layers = create_inference_model(model)
    # Encoder output which is the input initial state for the decoder
    encoder_output = encoder_model.predict(input_seqs)

    batch_size, no_decoder_tokens = input_seqs.shape[0], len(target_char_dec)

    # Generate empty decoder input sequence for 1 timestep
    decoder_input_seq = np.zeros((batch_size, 1))
    # Populate the input sequence with the start character at the 1st timestep
    decoder_input_seq[:, 0] = target_char_enc["\t"]

    states_value = encoder_output * no_dec_layers if type(encoder_output) is list else [encoder_output] * no_dec_layers
    # Sampling loop for a batch of sequences
    stop_mask = np.array([False] * batch_size)
    stop_condition = False
    decoded_words = [""] * batch_size
    iterations = 0
    while not stop_condition:
        decoder_output, *decoder_states = decoder_model.predict([decoder_input_seq] + states_value)

        # Sample the tokens
        sampled_tokens_enc = np.argmax(decoder_output[:, -1, :], axis=-1)
        sampled_chars = [target_char_dec[token] for token in sampled_tokens_enc]
        # Update stop_mask
        stop_mask = np.array([(stop_mask[i] or sampled_chars[i] == '\n') for i in range(batch_size)])
        # Update the decoded words
        decoded_words = [decoded_words[i]+sampled_chars[i] if not stop_mask[i] else decoded_words[i] for i in range(batch_size)]
        iterations += 1

        # Exit condition: either hit max number of timesteps in decoder or hit stop characters ("\n") in all the decoded words
        if np.all(stop_mask) or iterations > max_decoder_timesteps:
            stop_condition = True

        # Update the target sequence (of length 1).
        decoder_input_seq[:, 0] = sampled_tokens_enc

        # Update states
        states_value = decoder_states

    return decoded_words

def beam_decoder_infer(model, input_seqs, max_decoder_timesteps, K=1, target_seqs=None):
    # K : beam_width
    encoder_model, decoder_model, no_enc_layers, no_dec_layers = create_inference_model(model)
    # Encoder output which is the input initial state for the decoder
    encoder_output = encoder_model.predict(input_seqs)

    # Top K output sequences for each input 
    final_outputs = np.zeros((input_seqs.shape[0], K, max_decoder_timesteps), dtype=np.int32)
    # Errors for top K output sequences
    final_errors = np.zeros((input_seqs.shape[0], K))

    for b in range(input_seqs.shape[0]):
        encoder_out = encoder_output if type(encoder_output) is list else [encoder_output]
        encoder_out = [encoder_out[i][b:b+1] for i in range(len(encoder_out))]

        # Generate decoder input sequence for 1 timestep (and one sample)
        decoder_k_inputs = np.zeros((1, 1))
        # Populate the input sequence with the start character at the 1st timestep
        decoder_k_inputs[:, 0] = target_char_enc["\t"]
        # Generate decoder input sequences from it
        decoder_k_inputs = [decoder_k_inputs]

        # (log(probability) sequence, decoder output sequence) pairs. Probability starts with log(1) = 0
        decoder_k_outseqs = [(0, [])]
        # Categorical cross entropy error in the sequence
        errors = [0]

        states_values = [encoder_out * no_dec_layers]
        # Sampling loop for a batch of sequences
        for it in range(max_decoder_timesteps):
            All_k_beams = []
            All_decoder_states = []
            All_errors = []
            for k in range(len(decoder_k_outseqs)):
                decoder_output, *decoder_states = decoder_model.predict([decoder_k_inputs[k]] + states_values[k])

                # Top K scores
                top_k = np.argsort(decoder_output[-1, -1, :], axis=-1)[-K:]
                All_k_beams += [(
                    decoder_k_outseqs[k][0] + np.log(decoder_output[-1, -1, top_k[i]]),
                    decoder_k_outseqs[k][1] + [top_k[i]]
                ) for i in range(K)]
                
                if target_seqs is not None:
                    All_errors += [errors[k] - np.log(decoder_output[-1, -1, target_seqs[b, it]])] * K

                All_decoder_states += [decoder_states] * K

            # Sort and choose top K with max probabilities
            sorted_ind = list(range(len(All_k_beams)))
            sorted_ind = sorted(sorted_ind, key = lambda ix: All_k_beams[ix][0])[-K:][::-1]
            # Choose the top K decoder output sequences till now
            decoder_k_outseqs = [All_k_beams[ind] for ind in sorted_ind]

            # Update the target sequence (of length 1 timestep)
            decoder_k_inputs = []
            for ind in sorted_ind:
                temp = np.zeros((1, 1))
                temp[:, 0] = All_k_beams[ind][1][-1]
                decoder_k_inputs.append(temp)

            # Update states
            states_values = [All_decoder_states[ind] for ind in sorted_ind]

            # Update errors
            if target_seqs is not None:
                errors = [All_errors[ind] for ind in sorted_ind]

        final_outputs[b] = np.array([decoder_k_outseqs[i][1] for i in range(K)])
        if target_seqs is not None:
            final_errors[b] = np.array(errors)

    return final_outputs, final_errors

def beam_decoder(model, input_seqs, target_seqs_onehot, max_decoder_timesteps, K=1):
    target_seqs = np.argmax(target_seqs_onehot, axis=-1)
    k_outputs, k_errors = beam_decoder_infer(model, input_seqs, max_decoder_timesteps, K, target_seqs)

    errors, correct = 0, 0
    for i in range(k_outputs.shape[0]):
        matches = np.sum(k_outputs[i] == target_seqs[i], axis=1)
        best_k = np.argmax(matches)
        errors += k_errors[i][best_k]
        correct += matches[best_k]

    return errors, correct/(k_outputs.shape[0] * k_outputs.shape[1])

def convert_to_word(pred_list, char_enc, char_dec):
    pred_word = ''
    for l in pred_list:
        # Stop word : '\n'
        if l == char_enc['\n']:
            break
        pred_word += char_dec[l]
    
    return pred_word

load_run = 'z5xoybdt'
api = wandb.Api()
run_prev = api.run('abisheks/assignment3/'+load_run)
prev_model_file = run_prev.file('model-best.h5').download(replace=True)
model = tf.keras.models.load_model(prev_model_file.name)
print([layer.name for layer in model.layers])
# predicted_words = infer(model, train_enc_input[:10], target_char_enc, target_char_dec, 21)
# k_pred, _ = beam_decoder_infer(model, train_enc_input[:10], 21, 5)
# for i in range(10):
#     print('-' * 60)
#     print('English word:', train_inp[i])
#     for j in range(5):
#         print(f'Translated word (possibility {j+1}):', convert_to_word(k_pred[i,j,:], target_char_enc, target_char_dec))
error, acc = beam_decoder(model, val_enc_input, val_dec_target, max_decoder_seq_length, 4)
print(error, acc)


['input_1', 'embedding', 'input_2', 'encoder_1', 'embedding_1', 'encoder_2', 'decoder_1', 'decoder_2', 'dense']


In [ ]:
# Hyperparameter choices to sweep 
sweep_config = {
    'name': 'RNN',
    'method': 'bayes',                   # Possible search : grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'inp_emb_size': {
            'values': [16, 32, 64, 256]
        },
        'no_enc_layers': {
            'values': [1, 2, 3]
        },
        'no_dec_layers': {
            'values': [1, 2, 3]
        },
        'hid_layer_size': {
            'values': [16, 32, 64, 256]
        },
        'cell_type': {
            'values': ['RNN', 'LSTM', 'GRU']
        },
        'dropout' :{
            'values': [0, 0.25, 0.4]
        },
        'optimizer': {
            'values': ['adam', 'nesterov']
        },
        'beam': {
            'values': [1, 5]
        }
    }
}

In [ ]:
def sweep_wrapper():
    # Wrapper function to call the CNN function for sweeping with different hyperparameters

    # Initialize a new wandb run
    run = wandb.init(config=config_1, reinit=True)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    wandb.run.name = f'ie_{config.inp_emb_size}_ne_{config.no_enc_layers}_de_{config.no_dec_layers}_ct_{config.cell_type}_dr_{config.dropout}'
    wandb.run.name += f'_da_{config.hid_layer_size}'
    wandb.run.save()
    print(wandb.run.name)

    modelA = RNN_train(config)
    run.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="abisheks", project="assignment3")
wandb.agent(sweep_id, lambda : sweep_wrapper())

In [12]:
chumma = ['a','a']*4
print(chumma)

['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']
